# Retrieval and Search

In this tutorial we will put our knowledge database into a search engine which will be used to feed our LLM

In [1]:
import os
import json

from minsearch import Index

In [2]:
documents_file = os.path.join(os.path.dirname(os.getcwd()), 'documents.json')

In [3]:
with open(documents_file, 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q = 'The course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
boost = {
    "question":3.0,
    "section:":0.5
}
results = index.search(
    query=q,
    filter_dict={'course':"data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
)

In [10]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Generatiion

In this section we will pass the documents returned by our search engine to our LLM and use it to generate and answer

In [11]:
from dotenv import load_dotenv
from openai import OpenAI
import vertexai
from vertexai.generative_models import GenerativeModel

load_dotenv()

PROJECT_ID = os.getenv('PROJECT_ID')
REGION = os.getenv('REGION')

In [12]:
# Course version (Open AI)
client  = OpenAI()
response = client.chat.completions.create(
    model='gpt4o',
    messages=[{"role":"user", "content":q}]
)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
# vertex
vertexai.init(
    project = PROJECT_ID,
    location = REGION
)
vertex_llm = GenerativeModel("gemini-1.5-pro-001")

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
}

responses = vertex_llm.generate_content(
    list(q),
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response.text, end="")

c:\Users\User\Documents\LLM_zoomcamp\workshop\llm-zoomcamp-3.10\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Please provide me with more context! I need more information to answer your question.  To figure out if you can still enroll, tell me:

* **What course are you interested in?**  (Give me the name or a brief description)
* **Where is the course offered?** (Is it a school, university, online platform, etc.?)

Once I have this information, I can help you find out if it's still possible to enroll. 😊 


In [ ]:
context = ""

for doc in results:
    context += f"section: {doc['question']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n \n"
    

In [ ]:
print(context)

section: Course - Can I still join the course after the start date?
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 
 
section: Course - Can I follow the course after it finishes?
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project. 
 
section: Course - When will the course start?
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will

In [ ]:
prompt_template = [
    """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

""",
f"""
QUESTION: {q}
""",
f"""
CONTENT: {context}
"""
]

In [ ]:
prompt = [element.strip() for element in prompt_template]

In [ ]:
for element in prompt:
    print(element)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.
QUESTION: The course has already started, can I still enroll?
CONTENT: section: Course - Can I still join the course after the start date?
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 
 
section: Course - Can I follow the course after it finishes?
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. 

In [ ]:
responses = vertex_llm.generate_content(
    prompt,
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response.text, end="")

Yes, you can still submit homework assignments even if you don't register. However, keep in mind that there are deadlines for final projects. 


## Cleaning up the RAG

In [ ]:
def search(query):
    boost = {
        "question":3.0,
        "section:":0.5
    }

    results = index.search(
        query=query,
        filter_dict={'course':"data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=10
    )

    context = ""

    for doc in results:
        context += f"section: {doc['question']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n \n"

    return context

In [30]:
def build_prompt(query, search_results):
    prompt_template = [
    """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

""",
f"""
QUESTION: {query}
""",
f"""
CONTENT: {search_results}
"""
]

    prompt = [element.strip() for element in prompt_template]

    return prompt

In [31]:
vertexai.init(
    project = PROJECT_ID,
    location = REGION
)
def llm(prompt):
    
    vertex_llm = GenerativeModel("gemini-1.5-pro-001")

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 0,
        "top_p": 0.95,
    }

    responses = vertex_llm.generate_content(
        prompt,
        generation_config=generation_config,
        stream=True,
    )

    for response in responses:
        print(response.text, end="")

In [ ]:
query = 'Can I join after the course has started?'
search_results = search(query)
prompt = build_prompt(query, search_results)
llm(prompt)

Yes, you can submit homework even without registering for the course. However, keep in mind the deadlines for final projects. 


In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    llm(prompt)

In [ ]:
rag('the course has already started, can I still join?')

Yes, you can join the course even if it has already started and submit the homework. However, keep in mind the deadlines for final projects. 


## Using Elastic Search

In [14]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

In [15]:
es_client.info()

ObjectApiResponse({'name': 'f8dc1bfb1b07', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'A0X8sWgLRnSCq5lyDpdEJw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [16]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} # Allowsto filter
        }
    }
}

index_name = "course-questions"
response = es_client.indices.create(index=index_name, body=index_settings)

response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [17]:
# We bring the documents to elastic search using tqdm progressbar
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

c:\Users\User\Documents\llm-zoomcamp-dbeta95\llm-zoomcamp-3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:48<00:00, 19.58it/s]


In [18]:
user_question = "I just discovered the course. Can I still join?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^3", "text", "section"], # The hat means three times more importance
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [20]:
response = es_client.search(index=index_name, body=search_query)

In [24]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [25]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [26]:
def elastich_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # The hat means three times more importance
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [27]:
elastich_search(user_question)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [28]:
def rag(query):
    search_results = elastich_search(query)
    prompt = build_prompt(query, search_results)
    llm(prompt)

In [32]:
rag(user_question)

c:\Users\User\Documents\llm-zoomcamp-dbeta95\llm-zoomcamp-3.10\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Yes, you can join the course even after the start date. You can submit homework without registering. 
